# Data: Separating and Uniting Columns

*Purpose*: Data is easiest to use when it is *tidy*. In fact, grama is specifically designed to use tidy data. Last time we learned how to pivot data, but data can be untidy in other ways. Pivoting helped us when data were locked up in the *column headers*: This time, we'll learn how to use *separate* and *unite* to deal with *cell values* that are untidy.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

# For assertion
from pandas.api.types import is_integer_dtype

# Separate and Unite


Sometimes data are presented in a *wide* format in order to fit many values on a page. You'll often see this form of data when looking at large datasets that have been prepared for a report (say, a UN population survey). For example, the following dataset is clearly not tidy:


In [ ]:
## NOTE: No need to edit
gr.df_make(
    country=["Afghanistan", "Albania", "Algeria"],
    population_1995=[17586073, 3357858, 29315463],
    population_1996=[18415307, 3341043, 29845208],
)

The year of each population measurement is "locked up" in the column names. However, pivoting this dataset only gets us so far:


In [ ]:
## NOTE: No need to edit
(
    gr.df_make(
        country=["Afghanistan", "Albania", "Algeria"],
        population_1995=[17586073, 3357858, 29315463],
        population_1996=[18415307, 3341043, 29845208],
    )
    >> gr.tf_pivot_longer(
        columns=["population_1995", "population_1996"],
        names_to="name",
        values_to="value",
    )
)


The column names are not simply cell values; they're a combination of both column names and cell values. To fully tidy this datset, we need to *separate* the names from the data.


## The `gr.tf_separate()` verb

The `gr.tf_separate()` verb allows us to separate *string-valued* columns on a target character. As a simple example, consider the following (silly) dataset:


In [ ]:
## NOTE: No need to edit
(
    gr.df_make(s=["A-123", "B-456", "C-789"])
)

If we wanted to separate the column `s` into two columns on the `-` character, we could do that with `gr.tf_separate()`.


In [ ]:
## NOTE: No need to edit
(
    gr.df_make(s=["A-123", "B-456", "C-789"])
    >> gr.tf_separate(
        column="s",
        sep="-",
        into=["letter", "numbers"],
    )
)

Note that we have two new columns with the separated values. We can use this approach to help tidy datasets.


### __q1__ Separate the `population` and `year`

Use `gr.tf_separate()` to separate the `population` and `year` values into their own columns.


In [ ]:
## TASK: Separate the population and year columns
df_pop = (
    gr.df_make(
        country=["Afghanistan", "Albania", "Algeria"],
        population_1995=[17586073, 3357858, 29315463],
        population_1996=[18415307, 3341043, 29845208],
    )
    >> gr.tf_pivot_longer(
        columns=["population_1995", "population_1996"],
        names_to="pop_year",
        values_to="value",
    )

)

## NOTE: No need to edit; use this to check your work
print(df_pop)

assert \
    "population" in df_pop.columns, \
    "df_pop does not have a `population` column"
assert \
    "year" in df_pop.columns, \
    "df_pop does not have a `year` column"
assert \
    df_pop.shape == (6, 4), \
    "df_pop is the wrong shape"


## The `gr.tf_unite()` verb

The `gr.tf_unite()` verb performs the inverse of `gr.tf_separate()`; `gr.tf_unite()` allows us to combine multiple string columns into a single column. For example, we can undo the separation you performed above:


In [ ]:
## NOTE: No need to edit
(
    df_pop
    >> gr.tf_unite(
        "population-year",
        DF.population,
        DF.year,
        sep="-",
    )
)

Note that we have to choose what to name the new united column, the set of columns to combine, and a separator character with the `sep` argument. While the example above is not terribly practical, there are many cases where we'll have many strings that we might like to join.


### __q2__ Unite the digits

Add the `area` code to the *front* of each `localnumber` to complete the following phone numbers. Provide the full number as the column `number`. Make sure to use the separator character `-`.


In [ ]:
## TASK:
df_numbers = (
    gr.df_make(
        localnumber=["255-2112", "255-4200", "867-5309"],
        area=["212", "650", "814"],
    )

)

## NOTE: Use this to check your work
print(df_numbers)

assert \
    "number" in df_numbers.columns, \
    "df_numbers does not have a `number` column"
assert \
    (df_numbers >> gr.tf_filter(gr.str_detect(DF.number, "_"))).shape[0] == 0, \
    "Separator `_` detected; make sure to use `-` as your separator"
assert \
    (df_numbers >> gr.tf_filter(gr.str_detect(DF.number, "^212|^650|^814"))).shape[0] == 3, \
    "The area codes are not at the *front* of the numbers"


# Case Study: Tidying an Alloy Dataset

Now let's make use of `gr.tf_separate()` in tidying a more engineering-focused dataset. The following is a small dataset of measured aluminum alloy material properties.


In [ ]:
## NOTE: No need to edit
df_alloys = gr.df_make(
    thick=[0.022]*2 + [0.032]*2,
    E_00=[10600, 10600, 10400, 10300],
    mu_00=[0.321, 0.323, 0.329, 0.319],
    E_45=[10700, 10500, 10400, 10500],
    mu_45=[0.329, 0.331, 0.318, 0.326],
)

df_alloys

Note that this dataset is not tidy: The digits in the column names are the angles at which the alloys were tested. In order to tidy this dataset, we'll have to go through a few steps:


### __q3__ Add a unique observation identifier

Later, when we attempt a `gr.tf_pivot_wider()` down the line, we'll need a unique identifier for each row in the original dataset. Create a new column `id_obs` that uniquely identifies each row.

*Hint*: You could just use the DataFrame's `index` column to do this....


In [ ]:
## TASK: Add a unique identifier (UID) to each row
df_alloys_id = (
    df_alloys
    ## TODO: Add a UID column, call it `id_obs`

)

## NOTE: Use this to check your work
print(df_alloys_id)

assert \
    "id_obs" in df_alloys_id.columns, \
    "df_alloys_id has no `id_obs` column"

assert \
    len(set(df_alloys_id.id_obs)) == len(df_alloys_id.id_obs), \
    "The values in df_alloys_id.id_obs are not unique"

### __q4__ Separate the `variable` and `angle` data

The following code pivots the data longer; use `gr.tf_separate()` to separate the `variable` and `angle` data.


In [ ]:
## TASK: Separate the `variable` and `angle` data
df_alloys_separated = (
    df_alloys_id
    >> gr.tf_pivot_longer(
        columns=["E_00", "E_45", "mu_00", "mu_45"],
        names_to="variable_angle",
        values_to="value"
    )

)

## NOTE: use this to check your work
print(df_alloys_separated)

assert \
    "variable" in df_alloys_separated.columns, \
    "df_alloys_separated has no `variable` column"

assert \
    "angle" in df_alloys_separated.columns, \
    "df_alloys_separated has no `angle` column"


### __q5__ Finish tidying the data

Finish tidying the data: Use a pivot to move the `variable` names back to being column names, and make sure the `angle` values are integers.


In [ ]:
## TASK: Finish tidying the data
df_alloys_tidy = (
    df_alloys_separated

)

## NOTE: Use this to check your work
print(df_alloys_tidy)

assert \
    "E" in df_alloys_tidy.columns, \
    "df_alloys_tidy does not have an `E` column"
assert \
    "mu" in df_alloys_tidy.columns, \
    "df_alloys_tidy does not have an `mu` column"
assert \
    is_integer_dtype(df_alloys_tidy.angle), \
    "df_alloys_tidy.angle is not an integer column"


## Aside: What happens without the unique id?

Above, I made a fuss about adding a unique id column to the dataset before we start pivoting. What happens if we don't do that?


In [ ]:
# NOTE: Uncomment the code below and run this cell to see the error
# (
#     df_alloys
#     # NOTE: I didn't add a UID here; let's see what happens....
    
#     >> gr.tf_pivot_longer(
#         columns=["E_00", "E_45", "mu_00", "mu_45"],
#         names_to="varang",
#         values_to="value"
#     )
#     >> gr.tf_separate(
#         column="varang",
#         sep="_",
#         into=["var", "ang"],
#     )
#     >> gr.tf_pivot_wider(
#         names_from="var",
#         values_from="value",
#     )
# )

We had to do a *lot* of work to tidy this dataset. Perhaps there's a better way?


## A *magic* option: `".value"`

To help minimize some of the work necessary for tidying, `gr.tf_pivot_longer()` provides some special functionality. To illustrate this, let's take a look at a full tidying effort on a dataset similar to the alloy one:


In [ ]:
## NOTE: No need to edit, this demonstrates the manual approach we've already seen
(
    gr.df_make(
        A_01=[1,2,3,4],
        A_02=[5,6,7,8],
        B_01=[1,3,5,7],
    )
    # Generate a unique observation id
    >> gr.tf_mutate(uid=DF.index)
    # Pivot longer to turn column names into cell values
    >> gr.tf_pivot_longer(
        columns=["A_01", "A_02", "B_01"],
        names_to="varidx",
        values_to="value",
    )
    # Separate the "true" column names from the cell entries
    >> gr.tf_separate(
        column="varidx",
        sep="_",
        into=["var", "idx"],
    )
    # Pivot wider to place correct column names in header
    >> gr.tf_pivot_wider(
        names_from="var",
        values_from="value",
    )
)

This gets the job done. However, there is a much more succinct way to tidy this dataset using just one call to `gr.tf_pivot_longer()` with the `".value"` argument.


In [ ]:
## NOTE: No need to edit, this demonstrates the ".value" approach
(
    gr.df_make(
        A_01=[1,2,3,4],
        A_02=[5,6,7,8],
        B_01=[1,3,5,7],
    )
    
>> gr.tf_pivot_longer(
        columns=["A_01", "A_02", "B_01"],
        names_to=[".value", "idx"],
        names_sep="_",
    )
)

This style of calling `gr.tf_pivot_longer()` with the `names_sep` argument effectively *embeds* a call to `gr.tf_separate()` within the pivot. The verb then automates the tedious steps of creating a unique ID and pivoting wider.

In order to use this special functionality, we have to target the final column names using the special `".value"` argument. Note that this shows up in the `names_to` set of arguments; where we would have specified the name of the column that we would eventually pivot wider:

```python
    names_to=["var", "idx"],
    names_sep="_",
```

we instead flag this column as the target to pivot wider with the special argument

```python
    names_to=[".value", "idx"],
    names_sep="_",
```


### __q6__ Use the `".value"` approach

Use the `".value"` approach to tidy the alloy dataset in one pivot.


In [ ]:
## TASK: Use the ".value" approach to tidy the alloy dataset in one pivot
(
    gr.df_make(
        thick=[0.022]*2 + [0.032]*2,
        E_00=[10600, 10600, 10400, 10300],
        mu_00=[0.321, 0.323, 0.329, 0.319],
        E_45=[10700, 10500, 10400, 10500],
        mu_45=[0.329, 0.331, 0.318, 0.326],
    )

)
